In [1]:
import os, sys

notebook_dir = os.getcwd()

#add path
sys.path.append(os.path.abspath(os.path.join(notebook_dir, '..')))

In [2]:
from scripts.preprocess_data import *

In [3]:
preprocess_data("../telegram_messages.json")

Preprocessed data saved to preprocessed_data.csv


In [2]:
from scripts.CoNLL_formatting import *

In [3]:
# Read the data and save the subset_df in a variable
data = 'preprocessed_data.csv'
subset_df = read_subset_df(data)

# Call the label_data function to process and save the labeled data
label_data(subset_df)

         channel         sender                  timestamp  \
0  @ZemenExpress -1001307493052  2025-01-15 05:43:58+00:00   
1  @ZemenExpress -1001307493052  2025-01-15 05:43:58+00:00   
2  @ZemenExpress -1001307493052  2025-01-14 13:29:49+00:00   
3  @ZemenExpress -1001307493052  2025-01-14 13:29:49+00:00   
4  @ZemenExpress -1001307493052  2025-01-14 08:17:05+00:00   

                                             content  \
0                                                NaN   
1  💥💥..........................💥💥\n              ...   
2                                                NaN   
3  💥💥...................................💥💥\n\n📌Fe...   
4                                                NaN   

                                              tokens  
0                                                 []  
1  ['💥💥..........................💥💥', 'መልካም', 'በአ...  
2                                                 []  
3  ['💥💥...................................💥💥', '📌...  
4              